# Imports

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, LogisticRegression
# from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import seaborn as sn
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
import nltk
from sklearn.metrics import roc_auc_score, f1_score
import seaborn as sn
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error #add rmse


/Users/yananikolova/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/Users/yananikolova/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


# Reading files and making it regressionable

In [31]:
df_train = pd.read_csv('merged_cleaned_sentiment_train.csv').drop(['pos','neg','neu', 'compound'], axis = 1)
df_val = pd.read_csv('merged_cleaned_sentiment_validation.csv').drop(['pos','neg','neu', 'compound'], axis = 1)

In [32]:
df_train = pd.concat([df_train, pd.read_csv('lyrics_features_train.csv')], axis=1)
df_val = pd.concat([df_val, pd.read_csv('lyrics_features_val.csv')], axis=1)

In [33]:
df_train = pd.concat([df_train, pd.get_dummies(df_train.key, drop_first = True, prefix = 'key')], axis=1)
df_val = pd.concat([df_val, pd.get_dummies(df_val.key, drop_first = True, prefix = 'key')], axis=1)

In [34]:
df_train = df_train.drop(['Unnamed: 0', 'artist', 'trackname', 'id', 'time_signature', 'lyrics', 'lyrics_cleaned'], axis=1)
df_val = df_val.drop(['Unnamed: 0', 'artist', 'trackname', 'id', 'time_signature', 'lyrics', 'lyrics_cleaned'], axis=1)

In [35]:
df_train = df_train.dropna()
df_val = df_val.dropna()

In [38]:
df_train['key']

0        6
1        5
2        5
3        9
4        9
        ..
7590     2
7591     2
7592    11
7593     1
7594     6
Name: key, Length: 7595, dtype: int64

In [11]:
# function to get cross validation scores
def get_cv_scores(model, X_train, y_train):
    scores = cross_val_score(model,
                             X_train,
                             y_train,
                             cv=5,
                             scoring='r2')
    
    print('CV Mean: ', np.mean(scores))
    print('STD: ', np.std(scores))

# Splitting the data into X and y

In [36]:
#     train set
X_train = df_train.drop(['y_valence', 'y_arousal'], axis=1).values
y_train_valence = df_train.y_valence.values 
y_train_arousal = df_train.y_arousal.values
    
#     validation set
X_val = df_val.drop(['y_valence', 'y_arousal'], axis=1).values
y_val_valence = df_val.y_valence.values 
y_val_arousal = df_val.y_arousal.values 

In [26]:
df_train = df_train.drop(['y_valence', 'y_arousal'], axis=1)

In [72]:
def do_regression(X, y_1, y_2, X_validation, y_1_validation, y_2_validation, optimize = False):
    

    # parameters
    param_grid = {'fit_intercept':[True,False], 'positive':[True, False]}
    
    # Initialize model for Grid search
    lr_val = LinearRegression()
    lr_arou = LinearRegression()
    
    # Grid search
    clf_vale = GridSearchCV(lr_val, 
                            param_grid, 
                            scoring='r2', 
                            verbose=1, 
                            n_jobs=-1,
                            return_train_score=True)
    
    clf_arou = GridSearchCV(lr_arou, 
                            param_grid, 
                            scoring='r2', 
                            verbose=1, 
                            n_jobs=-1,
                            return_train_score=True)

    # Print best results on training data    
    clf_vale.fit(X, y_1)
    clf_arou.fit(X, y_2)
    
    # Print best results on training data
    # add new lines to separate rows
    print()
    print("Best parameter for Valence (CV score=%0.3f):" % clf_vale.best_score_)
    print(clf_vale.best_params_)
    
    print()
    print("Best parameter for Arousal (CV score=%0.3f):" % clf_arou.best_score_)
    print(clf_arou.best_params_)
    print()


    #Initialize models with best parameters
    lr_val_top = LinearRegression(fit_intercept=clf_vale.best_params_['fit_intercept'],  
                                    positive = clf_vale.best_params_['positive'])
    lr_arou_top = LinearRegression(fit_intercept=clf_arou.best_params_['fit_intercept'],  
                                    positive = clf_arou.best_params_['positive'])

    # get cross val scores for models 
    get_cv_scores(lr_val_top, X, y_1)
    get_cv_scores(lr_arou_top, X, y_2)


    #fit optimal models to train data 
    lr_val_fit = lr_val_top.fit(X, y_1)
    lr_arou_fit = lr_arou_top.fit(X, y_2)
    
    # validation scores 
    r2_validation_valence = lr_val_fit.score(X_validation, y_1_validation)
    r2_validation_arousal = lr_arou_fit.score(X_validation, y_2_validation)
    
    print()
    print(f'Validation score for Valence: {r2_validation_valence}')
    print(f'Validation score for Arousal: {r2_validation_arousal}')
    
    return clf_vale.best_params_, clf_arou.best_params_

In [56]:
def do_forest_regression(X, y_1, y_2, X_validation, y_1_validation, y_2_validation):
    
    # Initialize models
    rf_val = RandomForestRegressor(random_state=0)
    rf_arou = RandomForestRegressor(random_state=0)
    
    param_grid = { 
    'n_estimators': [100, 500],
    'max_depth' : [5,10, 15]
    }

    # Grid search
    clf_vale = GridSearchCV(rf_val, 
                            param_grid, 
                            scoring='r2', 
                            verbose=1, 
                            n_jobs=-1,
                            return_train_score=True)
    
    clf_arou = GridSearchCV(rf_arou, 
                            param_grid, 
                            scoring='r2', 
                            verbose=1, 
                            n_jobs=-1,
                            return_train_score=True)

    # Print best results on training data    
    clf_vale.fit(X, y_1)
    clf_arou.fit(X, y_2)

    # Print best results on training data
    print()
    print("Best parameter for Valence (CV score=%0.3f):" % clf_vale.best_score_)
    print(clf_vale.best_params_)
    
    print()
    print("Best parameter for Arousal (CV score=%0.3f):" % clf_arou.best_score_)
    print(clf_arou.best_params_)


    #Initialize models with best parameters
    rf_val_top = RandomForestRegressor(n_estimators = clf_vale.best_params_['n_estimators'], 
                                        max_depth = clf_vale.best_params_['max_depth'], random_state=0)
    rf_arou_top = RandomForestRegressor(n_estimators = clf_arou.best_params_['n_estimators'], 
                                        max_depth = clf_arou.best_params_['max_depth'], random_state=0)
    

    # get cross val scores
    get_cv_scores(rf_val_top, X, y_1)
    get_cv_scores(rf_arou_top, X, y_2)

    rf_val_fit = rf_val_top.fit(X, y_1)
    rf_arou_fit = rf_arou_top.fit(X, y_2)
    
    r2_validation_valence = rf_val_fit.score(X_validation, y_1_validation)
    r2_validation_arousal = rf_arou_fit.score(X_validation, y_2_validation)

    print()
    print(f'Validation score for Valence: {r2_validation_valence}')
    print(f'Validation score for Arousal: {r2_validation_arousal}')
    
    
    return clf_vale.best_params_, clf_arou.best_params_

In [46]:
def do_svr(X, y_1, y_2, X_validation, y_1_validation, y_2_validation):

    #Standardizing
    scaler = StandardScaler().fit(X)
    X = scaler.transform(X)
    X_validation = scaler.transform(X_validation)

    # Train model
    svr_val = SVR()
    svr_arou = SVR()
    
    param_grid = {'kernel' : ('linear', 'rbf'),
                  'C' : [1,5,10]
                 }

    # Grid search
    clf_vale = GridSearchCV(svr_val, 
                            param_grid, 
                            scoring='r2', 
                            verbose=1, 
                            n_jobs=-1,
                            return_train_score=True)
    
    clf_arou = GridSearchCV(svr_arou, 
                            param_grid, 
                            scoring='r2', 
                            verbose=1, 
                            n_jobs=-1,
                            return_train_score=True)

    # Print best results on training data    
    clf_vale.fit(X, y_1)
    clf_arou.fit(X, y_2)
    
    # Print best results on training data
    # add new lines to separate rows
    print()
    print("Best parameter for Valence (CV score=%0.3f):" % clf_vale.best_score_)
    print(clf_vale.best_params_)
    
    print()
    print("Best parameter for Arousal (CV score=%0.3f):" % clf_arou.best_score_)
    print(clf_arou.best_params_)
    print()


    # Train model
    svr_val_top = SVR(kernel = clf_vale.best_params_['kernel'], C = clf_vale.best_params_['C'], 
                        degree= clf_vale.best_params_['degree'], coef0 = clf_vale.best_params_['coef0'],
                        gamma = clf_vale.best_params_['gamma'])
    svr_arou_top = SVR(kernel = clf_arou.best_params_['kernel'], C = clf_arou.best_params_['C'], 
                        degree= clf_arou.best_params_['degree'], coef0 = clf_arou.best_params_['coef0'],
                        gamma = clf_arou.best_params_['gamma'])


    # get cross val scores
    get_cv_scores(svr_val_top, X, y_1)
    get_cv_scores(svr_arou_top, X, y_2)
    
    r2_validation_valence = svr_val_top.score(X_validation, y_1_validation)
    r2_validation_arousal = svr_arou_top.score(X_validation, y_2_validation)
    
    print()
    print(f'Validation score for Valence: {r2_validation_valence}')
    print(f'Validation score for Arousal: {r2_validation_arousal}')


    return clf_vale.best_params_, clf_arou.best_params_

In [37]:
def do_mlp(X, y_1, y_2, X_validation, y_1_validation, y_2_validation):

    # Normalization
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    X_validation = scaler.fit_transform(X_validation)

    # Initialize model
    mlp_val = MLPRegressor(random_state = 2)
    mlp_arou = MLPRegressor(random_state = 2)
    
    param_grid = {'hidden_layer_sizes':[(5), (10), (15), (5,5), (10,10), (15,15), (5,5,5), (10,10,10), (15,15,15)], 
    'max_iter':[500, 1000, 2000, 2500]}

    # Grid search
    clf_vale = GridSearchCV(mlp_val, 
                            param_grid, 
                            scoring='r2', 
                            verbose=1, 
                            n_jobs=-1,
                            return_train_score=True)
    
    clf_arou = GridSearchCV(mlp_arou, 
                            param_grid, 
                            scoring='r2', 
                            verbose=1, 
                            n_jobs=-1,
                            return_train_score=True)


    # Print best results on training data    
    clf_vale.fit(X, y_1)
    clf_arou.fit(X, y_2)
    
    # Print best results on training data
    # add new lines to separate rows
    print()
    print("Best parameter for Valence (CV score=%0.3f):" % clf_vale.best_score_)
    print(clf_vale.best_params_)
    
    print()
    print("Best parameter for Arousal (CV score=%0.3f):" % clf_arou.best_score_)
    print(clf_arou.best_params_)
    print()


    #Train model with best params
    mlp_val_top = MLPRegressor(hidden_layer_sizes=clf_vale.best_params_['hidden_layer_sizes'], max_iter=clf_vale.best_params_['max_iter'], random_state = 2)
    mlp_arou_top = MLPRegressor(hidden_layer_sizes=clf_arou.best_params_['hidden_layer_sizes'], max_iter=clf_arou.best_params_['max_iter'], random_state = 2)

    # get cross val scores 
    get_cv_scores(mlp_val_top, X, y_1)
    get_cv_scores(mlp_arou_top, X, y_2)

    mlp_val_fit = mlp_val_top.fit(X, y_1)
    mlp_arou_fit = mlp_arou_top.fit(X, y_2)
    
    r2_validation_valence = mlp_val_fit.score(X_validation, y_1_validation)
    r2_validation_arousal = mlp_arou_fit.score(X_validation, y_2_validation)
    
    print()
    print(f'Validation score for Valence: {r2_validation_valence}')
    print(f'Validation score for Arousal: {r2_validation_arousal}')

    
    return clf_vale.best_params_, clf_arou.best_params_

In [57]:
do_forest_regression(X_train, y_train_valence, y_train_arousal, X_val, y_val_valence, y_val_arousal)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Fitting 5 folds for each of 6 candidates, totalling 30 fits

Best parameter for Valence (CV score=0.133):
{'max_depth': 5, 'n_estimators': 500}

Best parameter for Arousal (CV score=0.185):
{'max_depth': 5, 'n_estimators': 500}
CV Mean:  0.13275661965988045
STD:  0.011784880327224306
CV Mean:  0.18475573779507143
STD:  0.02493407497896191


UnboundLocalError: local variable 'rf_arou_fit' referenced before assignment

In [59]:
rf_val_top = RandomForestRegressor(n_estimators = 500, 
                                        max_depth = 5, random_state=0)
rf_arou_top = RandomForestRegressor(n_estimators = 500, 
                                        max_depth = 5, random_state=0)

rf_val_fit = rf_val_top.fit(X_train, y_train_valence)
rf_arou_fit = rf_arou_top.fit(X_train, y_train_arousal)

r2_validation_valence = rf_val_fit.score(X_val, y_val_valence)
r2_validation_arousal = rf_arou_fit.score(X_val, y_val_arousal)

print()
print(f'Validation score for Valence: {r2_validation_valence}')
print(f'Validation score for Arousal: {r2_validation_arousal}')
    


Validation score for Valence: 0.13244754474966047
Validation score for Arousal: 0.22954186143541289


    Fitting 5 folds for each of 6 candidates, totalling 30 fits
    Fitting 5 folds for each of 6 candidates, totalling 30 fits

    Best parameter for Valence (CV score=0.133):
    {'max_depth': 5, 'n_estimators': 500}

    Best parameter for Arousal (CV score=0.185):
    {'max_depth': 5, 'n_estimators': 500}

    CV Mean:  0.13275661965988045
    STD:  0.011784880327224306
    
    CV Mean:  0.18475573779507143
    STD:  0.02493407497896191

    Validation score for Valence: 0.13244754474966047
    Validation score for Arousal: 0.22954186143541289 

### Linear Regression

In [14]:
do_regression(X_train, y_train_valence, y_train_arousal, X_val, y_val_valence, y_val_arousal)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Fitting 5 folds for each of 4 candidates, totalling 20 fits

Best parameter for Valence (CV score=0.111):
{'fit_intercept': True, 'positive': True}

Best parameter for Arousal (CV score=0.140):
{'fit_intercept': True, 'positive': True}

CV Mean:  0.11111028455728592
STD:  0.01146729431981076
CV Mean:  0.14019888233177344
STD:  0.016407509678998546

Validation score for Valence: 0.10736294104767685
Validation score for Arousal: 0.14409979172722753


({'fit_intercept': True, 'positive': True},
 {'fit_intercept': True, 'positive': True})

    Fitting 5 folds for each of 4 candidates, totalling 20 fits
    Fitting 5 folds for each of 4 candidates, totalling 20 fits

    Best parameter for Valence (CV score=0.110):
    {'fit_intercept': True, 'positive': True}

    Best parameter for Arousal (CV score=0.140):
    {'fit_intercept': True, 'positive': True}

    CV Mean:  0.10971270746043267
    STD:  0.011520359485346096
    CV Mean:  0.13982572692153966
    STD:  0.017046339734464865

    Validation score for Valence: 0.10470671333163195
    Validation score for Arousal: 0.1431543974033288
    ({'fit_intercept': True, 'positive': True},
    {'fit_intercept': True, 'positive': True})

In [67]:
df_train_partial = pd.read_csv('merged_cleaned_sentiment_train.csv')
df_train_partial = pd.concat([df_train_partial, pd.get_dummies(df_train.key, drop_first = True, prefix = 'key')], axis=1)
df_train_partial = df_train_partial.drop(['Unnamed: 0', 'artist', 'trackname', 'id', 'time_signature', 'lyrics', 'lyrics_cleaned', 'key'], axis=1)


df_val_partial = pd.read_csv('merged_cleaned_sentiment_validation.csv')
df_val_partial = pd.concat([df_val_partial, pd.get_dummies(df_train.key, drop_first = True, prefix = 'key')], axis=1)
df_val_partial = df_val_partial.drop(['Unnamed: 0', 'artist', 'trackname', 'id', 'time_signature', 'lyrics', 'lyrics_cleaned', 'key'], axis=1)

df_train_partial = pd.concat([df_train_partial, pd.read_csv('lyrics_features_train.csv').iloc[:, :-200]], axis=1).dropna()
df_val_partial = pd.concat([df_val_partial, pd.read_csv('lyrics_features_val.csv').iloc[:, :-200]], axis=1).dropna()

#     train set
X_train_partial = df_train_partial.drop(['y_valence', 'y_arousal'], axis=1).values
y_train_partial_valence = df_train_partial.y_valence.values 
y_train_partial_arousal = df_train_partial.y_arousal.values
    
#     validation set
X_val_partial = df_val_partial.drop(['y_valence', 'y_arousal'], axis=1).values
y_val_partial_valence = df_val_partial.y_valence.values 
y_val_partial_arousal = df_val_partial.y_arousal.values 


In [46]:
X_train_partial

(7596, 25)

In [68]:
df_train_partial.columns

Index(['danceability', 'energy', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       ...
       'tfidf_pca_141', 'tfidf_pca_142', 'tfidf_pca_143', 'tfidf_pca_144',
       'tfidf_pca_145', 'tfidf_pca_146', 'tfidf_pca_147', 'tfidf_pca_148',
       'tfidf_pca_149', 'tfidf_pca_150'],
      dtype='object', length=182)

In [69]:
do_regression(X_train_partial, y_train_partial_valence, y_train_partial_arousal, X_val_partial, y_val_partial_valence, y_val_partial_arousal)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Fitting 5 folds for each of 4 candidates, totalling 20 fits

Best parameter for Valence (CV score=0.164):
{'fit_intercept': True, 'positive': True}

Best parameter for Arousal (CV score=0.156):
{'fit_intercept': True, 'positive': True}

CV Mean:  0.16360860461294166
STD:  0.019479749276111142
CV Mean:  0.1555347105221878
STD:  0.015044826818414521

Validation score for Valence: 0.15323139981799105
Validation score for Arousal: 0.1747622664978824


({'fit_intercept': True, 'positive': True},
 {'fit_intercept': True, 'positive': True})

In [70]:
df_train_partial = pd.read_csv('merged_cleaned_sentiment_train.csv')
df_train_partial = pd.concat([df_train_partial, pd.get_dummies(df_train.key, drop_first = True, prefix = 'key')], axis=1).dropna()
df_train_partial = df_train_partial.drop(['Unnamed: 0', 'artist', 'trackname', 'id', 'time_signature', 'lyrics', 'lyrics_cleaned', 'key'], axis=1)


df_val_partial = pd.read_csv('merged_cleaned_sentiment_validation.csv')
df_val_partial = pd.concat([df_val_partial, pd.get_dummies(df_train.key, drop_first = True, prefix = 'key')], axis=1).dropna()
df_val_partial = df_val_partial.drop(['Unnamed: 0', 'artist', 'trackname', 'id', 'time_signature', 'lyrics', 'lyrics_cleaned', 'key'], axis=1)

#     train set
X_train_partial = df_train_partial.drop(['y_valence', 'y_arousal'], axis=1).values
y_train_partial_valence = df_train_partial.y_valence.values 
y_train_partial_arousal = df_train_partial.y_arousal.values
    
#     validation set
X_val_partial = df_val_partial.drop(['y_valence', 'y_arousal'], axis=1).values
y_val_partial_valence = df_val_partial.y_valence.values 
y_val_partial_arousal = df_val_partial.y_arousal.values 

In [73]:
do_regression(X_train_partial, y_train_partial_valence, y_train_partial_arousal, X_val_partial, y_val_partial_valence, y_val_partial_arousal)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Fitting 5 folds for each of 4 candidates, totalling 20 fits

Best parameter for Valence (CV score=0.174):
{'fit_intercept': True, 'positive': True}

Best parameter for Arousal (CV score=0.169):
{'fit_intercept': False, 'positive': False}

CV Mean:  0.1739597914628858
STD:  0.01904918788782633
CV Mean:  0.16927345119522438
STD:  0.02171195419566874

Validation score for Valence: 0.15811334175537117
Validation score for Arousal: 0.19883809186851342


({'fit_intercept': True, 'positive': True},
 {'fit_intercept': False, 'positive': False})

In [45]:
do_svr(X_train, y_train_valence, y_train_arousal, X_val, y_val_valence, y_val_arousal)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


/Users/yananikolova/opt/anaconda3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


KeyboardInterrupt: 

In [61]:
do_mlp(X_train, y_train_valence, y_train_arousal, X_val, y_val_valence, y_val_arousal)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Fitting 5 folds for each of 36 candidates, totalling 180 fits

Best parameter for Valence (CV score=0.129):
{'hidden_layer_sizes': (10, 10), 'max_iter': 500}

Best parameter for Arousal (CV score=0.158):
{'hidden_layer_sizes': 5, 'max_iter': 500}

CV Mean:  0.12862127613817412
STD:  0.014345325664449918
CV Mean:  0.15809361480244224
STD:  0.013992543325221827

Validation score for Valence: 0.06403035750427888
Validation score for Arousal: 0.19374567585584568


({'hidden_layer_sizes': (10, 10), 'max_iter': 500},
 {'hidden_layer_sizes': 5, 'max_iter': 500})

    Fitting 5 folds for each of 36 candidates, totalling 180 fits
    Fitting 5 folds for each of 36 candidates, totalling 180 fits

    Best parameter for Valence (CV score=0.129):
    {'hidden_layer_sizes': (10, 10), 'max_iter': 500}

    Best parameter for Arousal (CV score=0.158):
    {'hidden_layer_sizes': 5, 'max_iter': 500}

    CV Mean:  0.12862127613817412
    STD:  0.014345325664449918
    CV Mean:  0.15809361480244224
    STD:  0.013992543325221827

    Validation score for Valence: 0.06403035750427888
    Validation score for Arousal: 0.19374567585584568
    ({'hidden_layer_sizes': (10, 10), 'max_iter': 500},
    {'hidden_layer_sizes': 5, 'max_iter': 500})

### Feature Selection

In [16]:
from sklearn.feature_selection import RFE


In [84]:

for i in [5,10,15,20,25]:

    print(i, 'FEATURES')
    estimator = LinearRegression(fit_intercept= True, positive= True)
    selector = RFE(estimator, n_features_to_select=i, step=1)
    selector = selector.fit(X_train, y_train_valence)

    estimator2 = LinearRegression(fit_intercept= True, positive= True)
    selector2 = RFE(estimator2, n_features_to_select=i, step=1)
    selector_arou = selector2.fit(X_train, y_train_arousal)

    lr_valence = LinearRegression(fit_intercept= True, positive= True)
    lr_arousal = LinearRegression(fit_intercept= True, positive= True)
    get_cv_scores(estimator, X_train[:,selector.support_], y_train_valence)
    get_cv_scores(estimator, X_train[:,selector_arou.support_], y_train_arousal)

    print()

5 FEATURES
CV Mean:  0.1262451176791907
STD:  0.012233182074217735
CV Mean:  0.1511001193689173
STD:  0.018394987112202853

10 FEATURES
CV Mean:  0.12848109065258234
STD:  0.011630560289500033
CV Mean:  0.16234581950445054
STD:  0.01720335276747698

15 FEATURES
CV Mean:  0.1287407081579088
STD:  0.01136939381647846
CV Mean:  0.16269616662621028
STD:  0.01690105990297593

20 FEATURES
CV Mean:  0.13038716144873894
STD:  0.011933507116895078
CV Mean:  0.1617235276109967
STD:  0.016067900547081418

25 FEATURES
CV Mean:  0.1306106156990735
STD:  0.012173101528586356
CV Mean:  0.16129697710145624
STD:  0.015127846124592396



In [17]:
# 20 features
estimator = LinearRegression(fit_intercept= True, positive= True)
selector = RFE(estimator, n_features_to_select=20, step=1)
selector = selector.fit(X_train, y_train_valence)

estimator2 = LinearRegression(fit_intercept= True, positive= True)
selector2 = RFE(estimator2, n_features_to_select=20, step=1)
selector_arou = selector2.fit(X_train, y_train_arousal)

In [88]:
mlp_valence = MLPRegressor(hidden_layer_sizes= (10,10), max_iter=500)
mlp_arousal = MLPRegressor(hidden_layer_sizes= 5, max_iter=500)
mlp_val_fit = mlp_valence.fit(X_train[:,selector.support_], y_train_valence)
mlp_arou_fit = mlp_arousal.fit(X_train[:,selector_arou.support_], y_train_arousal)

r2_validation_valence = mlp_val_fit.score(X_val[:,selector.support_], y_val_valence)
r2_validation_arousal = mlp_arou_fit.score(X_val[:,selector_arou.support_], y_val_arousal)

In [89]:

print()
print(f'Validation score for Valence: {r2_validation_valence}')
print(f'Validation score for Arousal: {r2_validation_arousal}')


Validation score for Valence: 0.1188660608771629
Validation score for Arousal: 0.18742728449631385


In [29]:
df_train.columns[selector.support_]

Index(['danceability', 'valence', 'pos', 'neu', 'tfidf_pca_2', 'tfidf_pca_4',
       'tfidf_pca_6', 'tfidf_pca_8', 'tfidf_pca_34', 'tfidf_pca_47',
       'tfidf_pca_63', 'tfidf_pca_67', 'tfidf_pca_83', 'tfidf_pca_100',
       'tfidf_pca_111', 'tfidf_pca_119', 'tfidf_pca_121', 'tfidf_pca_136',
       'tfidf_pca_145', 'tfidf_pca_146'],
      dtype='object')

In [30]:
df_train.columns[selector2.support_]

Index(['energy', 'valence', 'pos', 'neg', 'neu', 'tfidf_pca_42',
       'tfidf_pca_47', 'tfidf_pca_49', 'tfidf_pca_60', 'tfidf_pca_62',
       'tfidf_pca_63', 'tfidf_pca_80', 'tfidf_pca_96', 'tfidf_pca_111',
       'tfidf_pca_114', 'tfidf_pca_115', 'tfidf_pca_123', 'tfidf_pca_124',
       'tfidf_pca_133', 'tfidf_pca_137'],
      dtype='object')

In [22]:
df_train.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11
0,0.320,0.798,6,-9.660,0,0.0584,0.00152,0.25100,0.0734,0.407,...,0,0,0,0,1,0,0,0,0,0
1,0.434,0.600,5,-10.784,0,0.0286,0.86700,0.45100,0.1110,0.753,...,0,0,0,1,0,0,0,0,0,0
2,0.530,0.270,5,-11.251,0,0.0365,0.91000,0.00000,0.3120,0.515,...,0,0,0,1,0,0,0,0,0,0
3,0.479,0.528,9,-10.104,0,0.1040,0.58000,0.00128,0.8140,0.185,...,0,0,0,0,0,0,0,1,0,0
4,0.639,0.893,9,-6.509,1,0.0360,0.28900,0.00816,0.2220,0.966,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
do_mlp(X_train, y_train_valence, y_train_arousal, X_val, y_val_valence, y_val_arousal)